In [1]:
import sys
sys.path.append('..')
sys.path.append('../ehrshot')
import copy
from pathlib import Path

from typing import Literal
import argparse
import pandas as pd
import numpy as np
import os

import torch
from torch import nn
from torch_uncertainty.routines import ClassificationRoutine
from torch_uncertainty.utils import TUTrainer
from lightning.pytorch import Trainer
from torch_uncertainty.models import deep_ensembles, mc_dropout
from torch_uncertainty.transforms import RepeatTarget
import torchvision.transforms as T

from torch.utils.data import TensorDataset, DataLoader
import torch.nn.functional as F
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import roc_auc_score
from tqdm import tqdm

In [2]:
results_dict = {}

labeling_functions=[
    "guo_los",
    "guo_readmission",
    "guo_icu",
    "new_hypertension",
    "new_hyperlipidemia",
    "new_pancan",
    "new_celiac",
    "new_lupus",
    "new_acutemi",
    "lab_thrombocytopenia",
    "lab_hyperkalemia",
    "lab_hyponatremia",
    "lab_anemia",
    "lab_hypoglycemia" # will OOM at 200G on `gpu` partition
]

unique_tasks_1 = ['guo_los', 'guo_readmission', 'guo_icu']
unique_tasks_2 = ['new_hypertension', 'new_hyperlipidemia', 'new_pancan', 'new_celiac', 'new_lupus', 'new_acutemi']
unique_tasks_3 = ['lab_thrombocytopenia', 'lab_hyperkalemia', 'lab_hyponatremia', 'lab_anemia', 'lab_hypoglycemia']

all_tasks = [unique_tasks_1, unique_tasks_2, unique_tasks_3]
all_tasks_name = ['unique_tasks_1', 'unique_tasks_2', 'unique_tasks_3']

In [3]:
class TwoLayerNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(TwoLayerNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.dropout1 = nn.Dropout(p=0.5)
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout1(x)
        x = F.relu(self.fc2(x))
        return x

def optim_recipe(model, lr_mult: float = 1.0):
    optimizer = torch.optim.SGD(model.parameters(), lr=0.05 * lr_mult)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)
    return {"optimizer": optimizer, "scheduler": scheduler}

max_epochs = 50
batch_size = 64

for i in tqdm(range(len(all_tasks))):
# for i in [0]:

    general_task_name = all_tasks_name[i]

    folder_path = f'multi_task_data_uq/{general_task_name}'

    train_x_name = os.path.join(folder_path, 'X_train_all.csv')
    train_y_name = os.path.join(folder_path, 'y_train_all.csv')
    val_x_name = os.path.join(folder_path, 'X_val_all.csv')
    val_y_name = os.path.join(folder_path, 'y_val_all.csv')

    X_train = pd.read_csv(train_x_name).to_numpy()
    y_train = pd.read_csv(train_y_name).to_numpy().reshape(-1)
    X_val = pd.read_csv(val_x_name).to_numpy()
    y_val = pd.read_csv(val_y_name).to_numpy().reshape(-1)

    # Create class weights
    class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
    class_weights = torch.tensor(class_weights, dtype=torch.float)

    X_train = torch.tensor(X_train).float()
    X_val = torch.tensor(X_val).float()

    y_train = torch.tensor(y_train).long()
    y_val = torch.tensor(y_val).long()

    # Create TensorDatasets
    train_dataset = TensorDataset(X_train, y_train)
    val_dataset = TensorDataset(X_val, y_val)

    # Create DataLoaders
    train_dl = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_dl = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    input_size = X_train.shape[1]
    hidden_size = 128
    num_classes = 2
    
    model = TwoLayerNet(input_size, hidden_size, num_classes)

    mc_model = mc_dropout(model, num_estimators=5, last_layer=False, on_batch=False)

    routine = ClassificationRoutine(
        num_classes=2,
        model=mc_model,
        loss=nn.CrossEntropyLoss(),
        optim_recipe=optim_recipe(mc_model),
        is_ensemble=True,
    )
    trainer = Trainer(accelerator="gpu", max_epochs=max_epochs, enable_progress_bar=False)

    trainer.fit(model=routine, train_dataloaders=train_dl, val_dataloaders=val_dl)

    current_all_tasks_list = all_tasks[i]
    for j in tqdm(range(len(current_all_tasks_list))):
        folder_path_test = f'multi_task_data_uq/{general_task_name}/{current_all_tasks_list[j]}'
        
        test_x_name = os.path.join(folder_path_test, 'X_test.csv')
        test_y_name = os.path.join(folder_path_test, 'y_test.csv')

        X_test = pd.read_csv(test_x_name).to_numpy()
        y_test = pd.read_csv(test_y_name).to_numpy().reshape(-1)

        X_test = torch.tensor(X_test).float()
        y_test = torch.tensor(y_test).long()

        test_dataset = TensorDataset(X_test, y_test)
        test_dl = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

        ens_perf = trainer.test(routine, dataloaders=[test_dl])
        
        results_dict[current_all_tasks_list[j]] = ens_perf

  0%|          | 0/3 [00:00<?, ?it/s]GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA A10G') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/ubuntu/anaconda3/lib/python3.11/site-packages/lightning/pytorch/core/optimizer.py:377: Found unsupported keys in the optimizer configuration: {'scheduler'}

  | Name                | Type             | Params | Mode 
-----------------------------------------------------------------
0 | model               | MCDropout        | 115 K  | train
1 | loss                | CrossEntropyLoss | 0      | train
2 | format_batch_fn     | Identity         |

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test/cal/ECE        │    0.10285300016403198    │
│       test/cal/aECE       │    0.09954403340816498    │
│       test/cls/Acc        │    0.7854214310646057     │
│      test/cls/Brier       │    0.3217475116252899     │
│       test/cls/NLL        │    0.5942076444625854     │
│     test/cls/entropy      │    0.2544362545013428     │
│   test/ens_Disagreement   │    0.12291570007801056    │
│     test/ens_Entropy      │    0.19257521629333496    │
│        test/ens_MI        │   0.061861034482717514    │
│       test/sc/AUGRC       │    0.06662537902593613    │
│       test/sc/AURC        │    0.09858623147010803    │
│    test/sc/CovAt5Risk     │            nan            │
│    test/sc/RiskAt80Cov    │    0.16230067610740662    │
└───────────────────────────┴───────────────────────────┘

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/ubuntu/anaconda3/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=63` in the `DataLoader` to improve performance.


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test/cal/ECE        │    0.07588839530944824    │
│       test/cal/aECE       │    0.07248623669147491    │
│       test/cls/Acc        │    0.8720877170562744     │
│      test/cls/Brier       │    0.21334438025951385    │
│       test/cls/NLL        │    0.4799403250217438     │
│     test/cls/entropy      │    0.1367439478635788     │
│   test/ens_Disagreement   │    0.08836677670478821    │
│     test/ens_Entropy      │    0.14871127903461456    │
│        test/ens_MI        │   0.046959370374679565    │
│       test/sc/AUGRC       │    0.03411272540688515    │
│       test/sc/AURC        │    0.05142194777727127    │
│    test/sc/CovAt5Risk     │    0.6007308959960938     │
│    test/sc/RiskAt80Cov    │    0.07876712083816528    │
└───────────────────────────┴───────────────────────────┘

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/ubuntu/anaconda3/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=63` in the `DataLoader` to improve performance.


...

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/ubuntu/anaconda3/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=63` in the `DataLoader` to improve performance.


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test/cal/ECE        │   0.010595527477562428    │
│       test/cal/aECE       │   0.010628064163029194    │
│       test/cls/Acc        │    0.8981171250343323     │
│      test/cls/Brier       │    0.14715009927749634    │
│       test/cls/NLL        │    0.24132278561592102    │
│     test/cls/entropy      │    0.21908196806907654    │
│   test/ens_Disagreement   │    0.0825161412358284     │
│     test/ens_Entropy      │    0.25144872069358826    │
│        test/ens_MI        │    0.03099474124610424    │
│       test/sc/AUGRC       │   0.017534291371703148    │
│       test/sc/AURC        │   0.021730227395892143    │
│    test/sc/CovAt5Risk     │    0.8392055630683899     │
│    test/sc/RiskAt80Cov    │    0.04154844954609871    │
└───────────────────────────┴───────────────────────────┘

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/ubuntu/anaconda3/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=63` in the `DataLoader` to improve performance.


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test/cal/ECE        │   0.004133633803576231    │
│       test/cal/aECE       │   0.004470176063477993    │
│       test/cls/Acc        │     0.986387312412262     │
│      test/cls/Brier       │   0.026593269780278206    │
│       test/cls/NLL        │    0.07051648944616318    │
│     test/cls/entropy      │   0.050741516053676605    │
│   test/ens_Disagreement   │   0.058698270469903946    │
│     test/ens_Entropy      │    0.19154928624629974    │
│        test/ens_MI        │   0.023497669026255608    │
│       test/sc/AUGRC       │   0.0037661800161004066   │
│       test/sc/AURC        │   0.006204493809491396    │
│    test/sc/CovAt5Risk     │            1.0            │
│    test/sc/RiskAt80Cov    │   0.008302778005599976    │
└───────────────────────────┴───────────────────────────┘

100%|██████████| 3/3 [31:02<00:00, 620.79s/it]


In [6]:
input_size

896

In [5]:
import json
with open('results_model_uq/results_multi_task_mc_dropout.json', 'w') as f:
    json.dump(results_dict, f)
current_all_tasks_list[j]

'lab_hypoglycemia'